In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [2]:
# Edit to point to your MIMIC directory.
dataDirStr = '/Users/gmessier/data/mimic-1.4/'

In [3]:
procedures_icd_df = pd.read_csv(dataDirStr + "PROCEDURES_ICD.csv")
procedures_icd_df.columns = procedures_icd_df.columns.str.lower()
procedures_icd_df

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331
...,...,...,...,...,...
240090,228330,67415,150871,5,3736
240091,228331,67415,150871,6,3893
240092,228332,67415,150871,7,8872
240093,228333,67415,150871,8,3893


`procedureevents_mv_df` contains procedures for patients, defined by ICD codes. The ICD codes are generated for billing purposes at the end of the hospital stay. ICD codes are recorded for all patient hospitalizations.

Refer to `D_ICD_PROCEDURES` table for exact definitions of each `icd9_code`.

In [4]:
print(f"There has been a total of 240095 procedures recorded, with {procedures_icd_df.subject_id.nunique()} unique patients.")

There has been a total of 240095 procedures recorded, with 42214 unique patients.


If a patient has had multiple procedures, `seq_num` will provide the order in which the procedures were performed.

`icd9_code` provides the ICD-9 code for the given procedure. This code can be joined to the `D_ICD_PROCEDURES` table.

In [5]:
c = procedures_icd_df.icd9_code.value_counts()[:5]
p =procedures_icd_df.icd9_code.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
3893,14731,6.14
9604,10333,4.30
966,9300,3.87
9671,9100,3.79
9904,7244,3.02
